In [6]:
from snowflake.snowpark import Session
import snowflake.connector

In [8]:
connection_parameters = {
    "account": "xe85544.east-us-2.azure",
    "user": "kx",
    "password": "Snowflake1",
    "role": "SYSADMIN",  # optional
    "warehouse": "KX",  # medium snowpark-optimized
    "database": "KX",
    "schema": "BRUNO",
}

session = Session.builder.configs(connection_parameters).create()

In [9]:
meta_df = session.sql("""
WITH COUNTED_SYMBOLS AS (
  SELECT "Symbol", COUNT(*) AS ROW_COUNT 
  FROM "KX"."BRUNO"."QUOTES" 
  GROUP BY "Symbol"
), 
BUCKETED_SYMBOLS AS (
  SELECT 
    "Symbol", 
    ROW_COUNT, 
    SUM(ROW_COUNT) OVER (ORDER BY ROW_COUNT DESC, "Symbol" ASC ROWS UNBOUNDED PRECEDING) AS CUM_ROW_COUNT
  FROM COUNTED_SYMBOLS
)

SELECT 
  "Symbol", 
  ROW_COUNT,
  FLOOR(CUM_ROW_COUNT / (SELECT FLOOR(SUM(ROW_COUNT) / 1000) FROM COUNTED_SYMBOLS)) + 1 AS BUCKET_NUMBER
FROM BUCKETED_SYMBOLS
ORDER BY BUCKET_NUMBER DESC
""").write.mode("overwrite").save_as_table("JOB_METADATA")

In [10]:
job_ids = list(session.table("JOB_METADATA").to_pandas()['BUCKET_NUMBER'].unique())
len(job_ids)

834

In [14]:
from datetime import datetime

conn = snowflake.connector.connect(
    account="xe85544.east-us-2.azure",
    user="kx",
    password='Snowflake1',
    role="SYSADMIN",  # optional
    warehouse="KX",  # medium snowpark-optimized
    database="KX",
    schema="BRUNO",
)

cur = conn.cursor()

start = datetime.now()

for i, job in enumerate(job_ids):
    cur.execute_async(f'CALL KX_PARALLEL({job})')
    print(job)
    
end = datetime.now()

str(end-start)

783
782
781
780
779
778
777
776
775
774
773
772
771
770
769
768
767
766
765
764
763
762
761
760
759
758
757
756
755
754
753
752
751
750
749
748
747
746
745
744
743
742
741
740
739
738
737
736
735
734
733
732
731
730
729
728
727
726
725
724
723
722
721
720
719
718
717
716
715
714
713
712
711
710
709
708
707
706
705
704
703
702
701
700
699
698
697
696
695
694
693
692
691
690
689
688
687
686
685
684
683
682
681
680
679
678
677
676
675
674
673
672
671
670
669
668
667
666
665
664
663
662
661
660
659
658
657
656
655
654
653
652
651
650
649
648
647
646
645
644
643
642
641
640
639
638
637
636
635
634
633
632
631
630
629
628
627
626
625
624
623
622
621
620
619
618
617
616
615
614
613
612
611
610
609
608
607
606
605
604
1000
993
992
991
990
989
988
987
952
951
950
949
948
947
946
945
944
943
942
941
940
939
938
937
936
935
934
933
932
931
930
929
928
927
926
925
924
923
922
921
996
995
994
998
997
603
602
601
600
599
598
597
596
595
594
593
592
591
590
589
588
587
586
585
584
583
582
581
580
579

'0:02:28.823624'

In [7]:
from datetime import datetime

start = datetime.now()

metadata = session.table("JOB_METADATA")
quotes = session.table("QUOTES")
trades = session.table("TRADES")

job_tickers = metadata.filter(F.col('BUCKET_NUMBER')==job)

quotes_df = quotes.join(job_tickers, quotes['"Symbol"']==job_tickers['"Symbol"']).select(
    quotes['"TTime"'],
    quotes['"Symbol"'].alias('"Symbol"'),
    quotes['"Bid_Price"'],
    quotes['"Offer_Price"']
).to_pandas()

trades_df = trades.join(job_tickers, trades['"Symbol"']==job_tickers['"Symbol"']).select(
    trades['"TTime"'],
    trades['"Symbol"'].alias('"Symbol"'),
    trades['"Trade Price"']
)

end = datetime.now()

str(end-start)

'0:00:06.074807'